[From Anton Poluektov's "Performing amplitude fits with `TensorFlow`" talk](https://indico.cern.ch/event/613842/contributions/2585806/attachments/1464213/2262841/tfa.pdf#page=6)

In [1]:
import tensorflow as tf
import numpy as np

import itertools # for fast looping
import time # for timing loop
from iminuit import Minuit # http://iminuit.readthedocs.io/en/latest/installation.html
from scipy.optimize import minimize

In [2]:
x = tf.placeholder(tf.float32, shape = (None))
w = tf.Variable(1.)
p = tf.Variable(0.)
a = tf.Variable(1.)

# Build the computational graph (a function is a graph!)
f = a * tf.sin(w*x + p)

In [3]:
def eval_func(f):
    
    with tf.Session() as sess:
        # Create TF session and initialise variables
        init = tf.global_variables_initializer()
        sess.run(init)
        
        # Run calculation of y by feeding data to tensor x
        return sess.run(f, feed_dict = { x : [1., 2., 3., 4.] })

In [4]:
y_data = eval_func(f)
print(y_data)

[ 0.84147096  0.90929741  0.14112    -0.7568025 ]


Other Examples

In [5]:
mu = 0.
sigma = 3.

# pdf of Gaussian of variable x with mean mu and standard deviation sigma
dist = tf.contrib.distributions.Normal(loc=mu, scale=sigma)

In [6]:
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    
    # Cumulative distribution funtion of pdf evalauted at x=1
    sess.run(dist.cdf(1.))
    
    # Evaluate the pdf at x=0
    sess.run(dist.prob(0.))
    
    sample_data = sess.run(dist.sample(10))
    
    y_data = sess.run(dist.prob(sample_data))

In [7]:
print(y_data)

[ 0.13263623  0.04962614  0.13274099  0.0158738   0.03045372  0.10307471
  0.08750544  0.02811804  0.13294983  0.125588  ]


In [8]:
def sample_model(model, n_samples):
    x = model.sample(n_samples)
    y = model.prob(x)
    
    with tf.Session() as sess:
        init = tf.global_variables_initializer()
        sess.run(init)
        return sess.run(x), sess.run(y)

In [9]:
test_x, test_y = sample_model(dist, 100)

In [10]:
print(test_x)
print(test_y)

[ 6.12136745  2.14929724 -2.42215586 -0.49874809  4.37878323 -1.2776072
 -0.28020611 -0.96697176  1.5103848  -2.01455927 -2.45346951  1.40705061
 -2.83250189  3.38850212 -2.29941034 -2.64653397 -1.43943262 -0.88001215
 -2.2763176  -1.82501221  2.12087607  0.95849395  0.19783127 -1.0628556
 -0.84167737  1.25285673 -3.36018062 -1.91486478 -1.58766961 -1.44333768
  1.58418798  3.36047673 -1.23031044  3.12290955 -0.8545562   3.02979612
 -0.2986058  -7.03484821  4.05515099 -1.72066498  4.70691252  1.95766425
  8.78167725  3.30249882  1.90468788 -0.74337786  3.6788187  -2.96089792
 -1.20180106  0.77169633 -2.56228232  1.92446089 -4.35904121  1.53148746
 -3.83214188  0.55638379  0.65219474 -2.75851107 -0.59912813  0.58427006
 -3.11324358 -6.84425354 -0.88985896  1.87047267 -3.32977295  4.44586849
 -2.37641382  2.33079553 -1.69873619 -0.3452386   2.26193571  0.53274149
  2.46531296  1.67562556  6.75644684 -3.26527333 -4.465343   -0.11815143
  0.98029894 -1.96178269 -3.96152592 -3.3050251   5.1

c.f. https://gist.github.com/ibab/45c3d886c182a1ea26d5

In [11]:
def normal_log(X, mu, sigma, TYPE=np.float32):
    return -tf.log(tf.constant(np.sqrt(2 * np.pi), dtype=TYPE) * sigma) - \
        tf.pow(X - mu, 2) / (tf.constant(2, dtype=TYPE) * tf.pow(sigma, 2))

In [12]:
def nll(X, mu, sigma, TYPE=np.float32):
    return -tf.reduce_sum(normal_log(X, mu, sigma, TYPE))

In [13]:
# MLE attempt
TYPE = np.float64

n_events = 1000000 # time of fit is very dependent on n_events
n_trials = 10

sess = tf.Session()

def func(mu_, sigma_):
    return sess.run(nll_, feed_dict={mu: mu_, sigma: sigma_})

# Gilles example
def func_scipy(x):
    return sess.run(nll_, feed_dict={mu: x[0], sigma: x[1]})


start_time = time.time()
for _ in itertools.repeat(None, n_trials):
    data = np.random.normal(0.5, 1.5, n_events).astype(TYPE)
    
    # Define data as a variable so that it will be cached
    X = tf.Variable(data, name='data')
    
    mu = tf.Variable(TYPE(1), name='mu')
    sigma = tf.Variable(TYPE(2), name='sigma')
    
    init = tf.global_variables_initializer()
    sess.run(init)
    
    nll_ = nll(X, mu, sigma, TYPE)
    
    # To guard against excessive output
    if n_trials > 1:
        print_level = 0
    else:
        print_level = 1
    
#     minuit = Minuit(func, mu_=10, sigma_=10, error_mu_=0.5, error_sigma_=0.5,
#            limit_mu_=(-1, 100), limit_sigma_=(0, 100), errordef=1, print_level=print_level)
#     minuit.migrad()
#     minuit.minos()
    
    # Gilles example
    ret = minimize(func_scipy, x0=[10, 10], bounds=[(-1, 100), (0.00001, 100)])
    #print(ret.x, ret.fun) # x is an array of fit values, fun is the value of the function passed
    
end_time = time.time()
time_duration = end_time - start_time
mean_fit_time = time_duration/n_trials

# mu_ = minuit.values['mu_']
# sigma_ = minuit.values['sigma_']

sess.close()

#print("mu = {}, sigma = {}".format(mu_, sigma_))
print("mu = {}, sigma = {}".format(ret.x[0], ret.x[1]))

print("\nfit {} times in {} seconds".format(n_trials, time_duration))
print("The average fit time is {} seconds".format(mean_fit_time))

mu = 0.4999729388987804, sigma = 1.499371662468684

fit 10 times in 5.992491960525513 seconds
The average fit time is 0.5992491960525512 seconds
